In [62]:
# imports
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import re
import sys, os

In [63]:
# pandas settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [68]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)


In [37]:
df = pd.read_csv("../data/raw/raw_data.csv")
warnings.filterwarnings("ignore")
print("Shape:", df.shape)



Shape: (5555, 16)


In [24]:
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

In [27]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

In [29]:
from IPython.display import display, HTML
display(HTML("<style>.dataframe td { text-align: right; }</style>"))


In [38]:
df.shape
df.info()
df.describe(include='all')
df.isna().sum().sort_values(ascending=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5555 entries, 0 to 5554
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Address                               5555 non-null   object
 1   City                                  5555 non-null   object
 2   Price                                 5555 non-null   object
 3   Lot size (m2)                         5555 non-null   object
 4   Living space size (m2)                5555 non-null   object
 5   Build year                            5471 non-null   object
 6   Build type                            5555 non-null   object
 7   House type                            5555 non-null   object
 8   Roof                                  5555 non-null   object
 9   Rooms                                 5555 non-null   object
 10  Toilet                                5555 non-null   object
 11  Floors                        

Position                                304
Estimated neighbourhood price per m2    169
Build year                               84
Garden                                   58
Lot size (m2)                             0
Address                                   0
Price                                     0
City                                      0
House type                                0
Build type                                0
Living space size (m2)                    0
Roof                                      0
Floors                                    0
Toilet                                    0
Rooms                                     0
Energy label                              0
dtype: int64

In [39]:
df[['Price', 'Lot size (m2)', 'Living space size (m2)', 'Rooms', 'Estimated neighbourhood price per m2']].head(10)


,Price,Lot size (m2),Living space size (m2),Rooms,Estimated neighbourhood price per m2
0,€ 525.000,251 m²,135 m²,5 kamers (4 slaapkamers),6.035
1,€ 425.000,181 m²,109 m²,4 kamers (3 slaapkamers),1.435
2,€ 575.000,198 m²,138 m²,5 kamers (4 slaapkamers),1.855
3,€ 259.500,231 m²,92 m²,5 kamers (3 slaapkamers),2.190
4,€ 1.050.000,423 m²,210 m²,6 kamers (5 slaapkamers),1.340
5,€ 445.000,127 m²,126 m²,5 kamers (4 slaapkamers),1.745
6,€ 425.000,170 m²,87 m²,5 kamers (3 slaapkamers),7.080
7,€ 825.000,413 m²,232 m²,7 kamers (6 slaapkamers),5.090
8,€ 249.500,235 m²,124 m²,5 kamers (4 slaapkamers),580
9,€ 550.000,137 m²,111 m²,4 kamers (3 slaapkamers),2.040


In [41]:
# Calculate the percentage of missing values for each column
# This reveals data quality issues quantitatively rather than visually
# Columns exceeding ~15–20% missing usually need special handling (drop, impute, or mark as 'Unknown')
(df.isna().sum() / len(df) * 100).round(2)

Address                                0.00
City                                   0.00
Price                                  0.00
Lot size (m2)                          0.00
Living space size (m2)                 0.00
Build year                             1.51
Build type                             0.00
House type                             0.00
Roof                                   0.00
Rooms                                  0.00
Toilet                                 0.00
Floors                                 0.00
Energy label                           0.00
Position                               5.47
Garden                                 1.04
Estimated neighbourhood price per m2   3.04
dtype: float64

In [42]:
# Inspect unique value samples in each key column to detect formatting issues or inconsistent entries
# Example: numeric columns stored as text ("€ 350.000" vs "350000")
# This helps design robust cleaning rules before type conversion
for col in ['Price', 'Lot size (m2)', 'Living space size (m2)', 'Rooms']:
    print(f"\n--- {col} ---")
    print(df[col].unique()[:10])


--- Price ---
['€ 525.000' '€ 425.000' '€ 575.000' '€ 259.500' '€ 1.050.000' '€ 445.000'
 '€ 825.000' '€ 249.500' '€ 550.000' '€ 859.000']

--- Lot size (m2) ---
['251 m²' '181 m²' '198 m²' '231 m²' '423 m²' '127 m²' '170 m²' '413 m²'
 '235 m²' '137 m²']

--- Living space size (m2) ---
['135 m²' '109 m²' '138 m²' '92 m²' '210 m²' '126 m²' '87 m²' '232 m²'
 '124 m²' '111 m²']

--- Rooms ---
['5 kamers (4 slaapkamers)' '4 kamers (3 slaapkamers)'
 '5 kamers (3 slaapkamers)' '6 kamers (5 slaapkamers)'
 '7 kamers (6 slaapkamers)' '9 kamers (7 slaapkamers)'
 '3 kamers (2 slaapkamers)' '4 kamers (2 slaapkamers)'
 '9 kamers (4 slaapkamers)' '6 kamers (4 slaapkamers)']


In [45]:
# Define a helper to clean numeric-like text fields
# Removes non-numeric symbols and converts to float for later numerical analysis

def to_numeric(value):
    if pd.isna(value):
        return None
    # Strip euro signs, commas, spaces, and non-digit characters
    cleaned = re.sub(r"[^\d.]", "", str(value))
    try:
        return float(cleaned)
    except ValueError:
        return None


In [46]:
# Apply numeric conversion to all relevant columns
numeric_cols = ['Price', 'Lot size (m2)', 'Living space size (m2)', 'Rooms', 'Estimated neighbourhood price per m2']
for col in numeric_cols:
    df[col] = df[col].apply(to_numeric)


In [48]:
# Summarize numeric distributions to verify plausibility (outliers, range errors, scale consistency)
# This ensures no corrupt values were introduced during cleaning
df.describe()


,Price,Lot size (m2),Living space size (m2),Rooms,Estimated neighbourhood price per m2
count,"5,168.00","5,555.00","5,555.00","5,555.00","5,386.00"
mean,487.98,256.15,146.43,58.92,102.55
std,178.75,201.60,62.29,43.18,244.64
min,149.00,1.00,53.00,2.00,1.00
25%,350.00,133.00,110.00,52.00,2.04
50%,450.00,197.00,130.00,54.00,3.45
75%,595.00,322.00,162.00,65.00,6.81
max,999.00,998.00,844.00,"1,816.00",995.00


In [49]:
(df.isna().sum() / len(df) * 100).round(2)

Address                                0.00
City                                   0.00
Price                                  6.97
Lot size (m2)                          0.00
Living space size (m2)                 0.00
Build year                             1.51
Build type                             0.00
House type                             0.00
Roof                                   0.00
Rooms                                  0.00
Toilet                                 0.00
Floors                                 0.00
Energy label                           0.00
Position                               5.47
Garden                                 1.04
Estimated neighbourhood price per m2   3.04
dtype: float64

In [51]:
df.loc[df['Rooms'] > 20, 'Rooms'] = None
df.loc[df['Estimated neighbourhood price per m2'] > 50, 'Estimated neighbourhood price per m2'] /= 100  # fix decimal scaling


In [70]:
import os
print(os.getcwd())


c:\Users\Kille\Documents\nl-housing\notebooks


ModuleNotFoundError: No module named 'src.data.transform.validate_schema'